
#**Jupyter Notebook Essentials: Analyzing migrant deaths** 

In this workshop, we will learn how to:
*   Navigate Jupyter notebooks
*   Open, filter, and plot data
*   Work with Markdown
*   Convert notebooks to Python scripts

We will access data from the **International Organization for Migration (IOM)** Missing Migrants Project, who tracks deaths of migrants who have gone missing along mixed migration routes worldwide.

*For those interested in learning more, below are some links:*

*   [IOM Missing Migrants](https://missingmigrants.iom.int/about)
*   [Missing in Brooks County Documentary](https://www.missinginbrookscounty.com/)
    * Funded in part by [The University of Arizona Office for Research, Innovation & Impact](https://research.arizona.edu/)
*   [The Line Becomes a River: Dispatches from the Border](http://ezproxy.library.arizona.edu/login?url=https://search.ebscohost.com/login.aspx?direct=true&db=nlebk&AN=1536371&site=ehost-live&ebv=EK&ppid=Page-__-1)



In [ ]:
!pip3 install geocoder

import sys 
import os
import pandas as pd
import numpy as np
import glob 
import subprocess as sp
from IPython.display import HTML
from datetime import datetime
import plotly.express as px
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import geocoder
import seaborn as sns

In [ ]:
def get_data():

    now = datetime.now()
    link = f'https://missingmigrants.iom.int/global-figures/all/xls/\
    MissingMigrants-Global-{now.date()}T{str(now.hour).zfill(2)}-\
    {str(now.minute).zfill(2)}-{str(now.second).zfill(2)}.xls'
    file_path = os.path.join('migrant_deaths_data.xls')

    if not os.path.isfile(file_path):

        sp.call(f'wget {link} -O {file_path}', shell=True)
    print(f'Data has been downloaded. See {file_path}')

    return file_path


#-------------------------------------------------------------------------------
def clean_df(df):

    df = df.drop(['Web ID', 'Reported Month',\
                  'Minimum Estimated Number of Missing'], axis=1)\
        .rename(columns={'Number Dead': 'num_dead', 'Total Dead and Missing':\
                         'total_dead_missing','Number of Survivors':\
                         'num_survivors', 'Number of Females': 'num_females', 
                         'Number of Males': 'num_males', 'Number of Children':\
                         'num_children', 'Cause of Death': 'cause_death',\
                         'Location Description': 'location_description',\
                         'Information Source': 'information_source',\
                         'Location Coordinates': 'location_coordinates', 
                         'Migration Route': 'migration_route',
                         'UNSD Geographical Grouping': 'geographical_grouping',
                         'Source Quality': 'source_quality', 
                         'Region': 'region', 'Reported Date': 'reported_date',
                         'Reported Year': 'reported_year'})

    df['reported_date'] = pd.to_datetime(df['reported_date'])
    df = df.convert_dtypes()
    return df


#-------------------------------------------------------------------------------
def sum_df_dates(df):

    sum_df = df.groupby(by=['reported_date', 'region', 'migration_route',\
                            'location_coordinates', 'geographical_grouping',\
                            'information_source', 'reported_year']).sum()
    return sum_df


#-------------------------------------------------------------------------------
def open_df(file_path):

    with open(file_path, 'r') as f:
        dfs = pd.read_html(f.read())

    df = dfs[0]
    return df


#-------------------------------------------------------------------------------
def filter_df(df, region=None, quality=None):

    if region:
        filtered_df = df[df['region']==region]
    if quality:
        filtered_df = filtered_df[filtered_df['source_quality']==quality]

    return filtered_df


#-------------------------------------------------------------------------------
def prepare_geocoords(df): 
    
    geo_df = df.reset_index()
    geo_df['lat'] = geo_df['location_coordinates'].str.split(', ', expand=True)[0]
    geo_df['lon'] = geo_df['location_coordinates'].str.split(', ', expand=True)[1]
    geo_df['num_dead'] = geo_df['num_dead'].astype(float) 
    geo_df['lat'] = geo_df['lat'].astype(float)
    geo_df['lon'] = geo_df['lon'].astype(float)

    return geo_df


#-------------------------------------------------------------------------------
def set_mapbox(api_key):

    px.set_mapbox_access_token(token=api_key)


#-------------------------------------------------------------------------------
def add_geographical_details(df):

    geolocator = Nominatim(user_agent="geoapiExercises")
    df['city']=df['county']=df['state']=df['country']=df['zip_code'] = None

    for i, row in df.iterrows(): 

        location = geolocator.reverse(str(row['lat'])+","\
                                    +str(row['lon']))
        
        address = location.raw['address']
        city = address.get('city', np.nan)
        county = address.get('county', np.nan)
        state = address.get('state', np.nan)
        country = address.get('country', np.nan)
        code = address.get('country_code', np.nan)
        zipcode = address.get('postcode', np.nan)

        df.at[i, 'city'] = city 
        df.at[i, 'county'] = county
        df.at[i, 'state'] = state
        df.at[i, 'country'] = country
        df.at[i, 'zip_code'] = zipcode
        
    return df

# Understanding the data/problem

In [ ]:
HTML('<iframe \
        width="1280" \
        height="720" \
        src="https://www.youtube.com/embed/pxwVSCgBOy0" \
        title="YouTube video player" \
        frameborder="0" \
        allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" \
        allowfullscreen></iframe>')

# **Open, filter, and plot data**
## Access migration death data

In [ ]:
file_path = get_data()
df = open_df(file_path)
df = clean_df(df)
sum_df = sum_df_dates(df)

## Plot international deaths

In [ ]:
fig = px.line(sum_df.reset_index(), 
              x="reported_date", 
              y="num_dead", 
              color='region', 
              line_group='region',
              labels={
                     "num_dead": "Number of Deaths",
                     "reported_date": "Reported Date"
                 },
              title='International migrant deaths').for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

fig.show()

## Filter data to include only US-Mexico deaths

In [ ]:
filtered_df = filter_df(df, region='US-Mexico Border', quality=5)
sum_df = sum_df_dates(filtered_df)

In [ ]:
sum_df.reset_index()['region'].unique()

In [ ]:
import plotly.express as px
fig = px.line(sum_df.reset_index(), 
              x="reported_date", 
              y="num_dead", 
              color='region',
              labels={
                     "num_dead": "Number of Deaths",
                     "reported_date": "Reported Date"
                 },
              title='US-Mexico migrant deaths').for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.show()

## What could have led to the spike in September 2014?

In [ ]:
%%html
<iframe src="https://www.onthisday.com/date/2014/september" style="background-color: Snow;" width="1200" height="1000"></iframe>

## December 2015?

In [ ]:
%%html
<iframe src="https://www.onthisday.com/date/2015/december" style="background-color: Snow;" width="1200" height="1000"></iframe>

## December 2019?

In [ ]:
%%html
<iframe src="https://www.onthisday.com/date/2019/december" style="background-color: Snow;" width="1200" height="1000"></iframe>

## **GIS**
### Based on the data and historical records, politics and epi/pan-demics seem to drive increases in migration. Now let's map deaths on a web map to find hot spots. 
Questions we can answer: 
*   What state has the most migrant deaths?
*   Are deaths more frequent/numerous at the border wall or within the US? 
*   Are deaths more frequent/numerous in incorporated or unincorporated areas?

In [ ]:
geo_df = prepare_geocoords(sum_df)

In [ ]:
set_mapbox(api_key='pk.eyJ1IjoiZW1tYW51ZWxnb256YWxleiIsImEiOiJja3RndzZ2NmIwbTJsMnBydGN1NWJ4bzkxIn0.rtptqiaoqpDIoXsw6Qa9lg')

In [ ]:
fig = px.scatter_mapbox(geo_df, 
                        lat="lat", 
                        lon="lon", 
                        color="information_source", 
                        size='num_dead', 
                        zoom=4, 
                        opacity=1,

                        mapbox_style='satellite-streets').for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

fig.update_layout(
        title = 'US-Mexico migrant deaths (2014-2021)',
        geo_scope='usa',
        autosize=True
)

fig.show()

## The code below will take about 8 minutes to run. Feel free to ask questions while we wait!

In [ ]:
geo_df = add_geographical_details(geo_df)

In [ ]:
px.bar(geo_df[geo_df['country']=='United States'].groupby(by=['state', 'county']).sum().reset_index(),
       x='state',
       y='num_dead',
       color='county',
       title='Number of deaths per State',
       labels={
            "num_dead": "Number of Deaths",
            "state": "State"}).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

In [ ]:
geo_df['year'] = geo_df['reported_date'].dt.year

px.line(geo_df[geo_df['state']=='Arizona'].groupby(by=['county', 'state', 'year']).sum().reset_index(),
        x='year',
        y='num_dead', 
        color='county',
        line_group='state',
        title='Number of deaths in Arizona by Year, County',
        labels={
            "num_dead": "Number of Deaths",
            "year": "Year"}).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))


In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook")
sns.set(rc={'figure.figsize':(9.7,6.27)})

sns.lineplot(x='year',
             y='num_males', 
             data=geo_df[geo_df['state']=='Arizona'].groupby(by=['state', 'year']).sum().reset_index())

sns.lineplot(x='year',
             y='num_females', 
             data=geo_df[geo_df['state']=='Arizona'].groupby(by=['state', 'year']).sum().reset_index())

plt.legend(['Male', 'Female'])
plt.xlabel('Year')
plt.ylabel('Number of deaths')
plt.title('Number of deaths by biological sex')
plt.tight_layout();